# CIFAR-10
This notebook shows some examples of compressions of models trained on the CIFAR-10 dataset.

# Setup

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import sys

sys.path.append('../')
sys.path.append('../src/')

import src.general as general
import src.dataset_models as data
import src.metrics as metrics
import src.evaluation as eval
import src.plot as plot
import src.compression.distillation as distill
import src.compression.pruning as prune


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


### Dataset & Model

In [3]:
dataset = data.supported_datasets["CIFAR-10"]
dataset.cap = 100

In [4]:
import models.trained_models_cifar10.models.resnet_models as rs


resnet = rs.ResNet50()
device = general.get_device()
state_dict = torch.load('../models/trained_models_cifar10/resnet50_cifar10_lr01.pth', map_location=device)
resnet.load_state_dict(state_dict["net"])

<All keys matched successfully>

In [5]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Bottleneck(
      (

In [32]:
prune.get_ignored_layers(resnet)

[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 Linear(in_features=2048, out_features=10, bias=True)]

### Evaluation
Evaluate the model before any compression

In [7]:
before_results = eval.get_results(resnet, dataset)
plot.print_results(**before_results)

Test: 100%|██████████| 157/157 [07:18<00:00,  2.79s/it]


Test loss: -10.7890
Test score: 95.2926
==================================== RESULTS ===================================
Loss: -10.789035
Score: 95.292596
Time per data point: 174.3914 ms
Model Size: 90.02 MB
Number of parameters: 23520842
Number of MACs: 1304694784


In [23]:
example_input = general.get_example_input(dataset.train_loader)
eval.get_MACS(resnet, example_input)

Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.


835498188.15625

# Compression

## Pruning

### Structured pruning example.

In [8]:
model = prune.magnitude_pruning_structured(resnet, dataset, sparsity=0.1, fineTune=False)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [9]:
general.test(model, dataset)

Test:  77%|███████▋  | 121/157 [04:49<01:27,  2.43s/it]

In [18]:
magnitude_pruned_results = eval.get_results(model, dataset)
plot.print_results(**before_results)

Test:   0%|          | 0/157 [00:00<?, ?it/s]

Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.


Test:   1%|          | 1/157 [00:02<05:25,  2.08s/it]

Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.


Test:   1%|▏         | 2/157 [00:04<05:12,  2.02s/it]

...

Test:  99%|█████████▉| 156/157 [05:08<00:02,  2.01s/it]

Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.


Test: 100%|██████████| 157/157 [05:09<00:00,  1.97s/it]

Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Test loss: -6.2949
Test score: 48.3877
Could not calculate FLOPS


Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
Op Sequential is not supported at now.
==================================== RESULTS ===================================
Loss: -10.791916
Score: 95.322452
Time per data point: 174.4601 ms
Model Size: 90.02 MB
Number of parameters: 23520842
Number of FLOPs: 130

In [19]:
plot.print_before_after_results(before_results, magnitude_pruned_results)

============================= RESULTS BEFORE & AFTER ===========================
Loss: -10.791916 -> -6.294905 (-41.67%)
Score: 95.322452 -> 48.387739 (-49.24%)
Time per data point: 174.4601 ms -> 123.2721 ms (-29.34%)
Model Size: 90.02 MB -> 57.53 MB (-36.09%)
Number of parameters: 23520842 -> 15016264 (-36.16%)
Number of FLOPs: 1304694784 -> -1 (-100.00%)
Number of MACs: 1304694784 -> 835498188 (-35.96%)


In [16]:
prune.magnitude_pruning_structured(resnet,dataset, sparsity=0.8, fineTune=True)

Train: 100%|██████████| 782/782 [05:24<00:00,  2.41it/s]


Train loss: -428658.7833
Train score: 27.2498


Train: 100%|██████████| 782/782 [03:41<00:00,  3.53it/s]


Train loss: -6314159519977.1807
Train score: 11.7587


Train: 100%|██████████| 782/782 [02:01<00:00,  6.45it/s]

Train loss: -68383201205360295936.0000
Train score: 9.9365


ResNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Distillation

In [18]:
settings = {
    # "performance_target": 99,
    "fineTune": False,
    "epochs": 3,
}

distilled_model = distill.perform_distillation(resnet, dataset, settings)

Settings: {'fineTune': False, 'epochs': 3}
============================= CREATING STUDENT MODEL ===========================
Fine-tuning: False


============================ PERFORMING DISTILLATION ===========================
Epoch: 0


Distillation Training:   1%|          | 5/782 [00:01<04:59,  2.59it/s]


KeyboardInterrupt: 

In [33]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Bottleneck(
      (